In [2]:
# Na bazie OpenF1Api via https://openf1.org

In [3]:
# wyznaczenie liczby sezonów - lata od tzw "ery hybrydowej" 2014-2024
def seasons_count(end_year=2024):
    start_year = 2014
    seasons = list()
    for year in range(start_year, end_year + 1):
        seasons.append(year)
    return seasons

seasons = seasons_count(2024)
print(seasons)

[2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]


In [3]:
# formatowanie pól daty i czasu

from datetime import datetime

def format_datetime_fields(datetime_string):
    dt = datetime.fromisoformat(datetime_string.replace('Z', '+00:00'))
    return dt.date().isoformat(), dt.time().isoformat()

In [4]:
# pobiernie meetiings - race week 
import requests
import pandas as pd

def get_all_meetings(start_season=2014, end_season=2024):
    meetings = []
    
    for year in range(start_season, end_season + 1):
        url = f"https://api.openf1.org/v1/meetings?year={year}"
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            
            if isinstance(data, list) and len(data) > 0:
                for meeting in data:
                    date_start, time_start = format_datetime_fields(meeting['date_start'])
                    meetings.append({
                        "circuit_key": meeting['circuit_key'],
                        "circuit_short_name": meeting['circuit_short_name'],
                        "country_code": meeting['country_code'],
                        "country_key": meeting['country_key'],
                        "country_name": meeting['country_name'],
                        "date_start": date_start,
                        "time_start": time_start,
                        "gmt_offset": meeting['gmt_offset'],
                        "location": meeting['location'],
                        "meeting_key": meeting['meeting_key'],
                        "meeting_name": meeting['meeting_name'],
                        "meeting_official_name": meeting['meeting_official_name'],
                        "year": meeting['year'],
                    })
                print(f"Fetched {len(data)} meetings for {year}")
            else:
                print(f"No meetings found for {year}")
        else:
            print(f"Error fetching data for {year}: {response.status_code}")
    
    return pd.DataFrame(meetings)

In [5]:
all_meetings_df = get_all_meetings(2014, 2024)
all_meetings_df.head()

No meetings found for 2014
No meetings found for 2015
No meetings found for 2016
No meetings found for 2017
No meetings found for 2016
No meetings found for 2017
No meetings found for 2018
No meetings found for 2019
No meetings found for 2018
No meetings found for 2019
No meetings found for 2020
No meetings found for 2021
No meetings found for 2020
No meetings found for 2021
No meetings found for 2022
Fetched 23 meetings for 2023
No meetings found for 2022
Fetched 23 meetings for 2023
Fetched 25 meetings for 2024
Fetched 25 meetings for 2024


,circuit_key,circuit_short_name,country_code,country_key,country_name,date_start,time_start,gmt_offset,location,meeting_key,meeting_name,meeting_official_name,year
0,63,Sakhir,BRN,36,Bahrain,2023-02-23,07:00:00,03:00:00,Sakhir,1140,Pre-Season Testing,FORMULA 1 ARAMCO PRE-SEASON TESTING 2023,2023
1,63,Sakhir,BRN,36,Bahrain,2023-03-03,11:30:00,03:00:00,Sakhir,1141,Bahrain Grand Prix,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2023,2023
2,149,Jeddah,KSA,153,Saudi Arabia,2023-03-17,13:30:00,03:00:00,Jeddah,1142,Saudi Arabian Grand Prix,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2023,2023
3,10,Melbourne,AUS,5,Australia,2023-03-31,01:30:00,11:00:00,Melbourne,1143,Australian Grand Prix,FORMULA 1 ROLEX AUSTRALIAN GRAND PRIX 2023,2023
4,144,Baku,AZE,30,Azerbaijan,2023-04-28,09:30:00,04:00:00,Baku,1207,Azerbaijan Grand Prix,FORMULA 1 AZERBAIJAN GRAND PRIX 2023,2023


In [6]:
meetings_key_unique = all_meetings_df['meeting_key'].unique()
print(meetings_key_unique)

[1140 1141 1142 1143 1207 1208 1210 1211 1212 1213 1214 1215 1216 1217
 1218 1219 1220 1221 1222 1223 1224 1225 1226 1228 1229 1230 1231 1232
 1233 1234 1235 1236 1237 1238 1239 1240 1241 1242 1243 1244 1245 1246
 1247 1248 1249 1250 1251 1252]


In [7]:
# pobranie poszczególnych wyścigów w określonych sezonach - sessions
import requests
import pandas as pd
from time import sleep

def get_all_sessions(meetings):
    sessions = []
    
    for meeting in meetings_key_unique:
        url = f"https://api.openf1.org/v1/sessions?meeting_key={meeting}"
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            
            if isinstance(data, list) and len(data) > 0:
                for session in data:
                    # Parse dates and times
                    date_start, time_start = format_datetime_fields(session['date_start'])
                    date_end, time_end = format_datetime_fields(session['date_end'])
                    
                    sessions.append({
                        'meeting_key': session['meeting_key'],
                        'year': session['year'],
                        'session_key': session['session_key'],
                        'session_type': session['session_type'],
                        'session_name': session['session_name'],
                        'date_start': date_start,
                        'time_start': time_start,
                        'date_end': date_end,
                        'time_end': time_end,
                        'gmt_offset': session['gmt_offset'],
                        'location': session['location'],
                        'country_name': session['country_name'],
                        'country_code': session['country_code'],
                        'circuit_short_name': session['circuit_short_name'],
                    })
                
                print(f"Fetched {len(data)} sessions for {meeting}")
            else:
                print(f"No sessions found for {meeting}")
        else:
            print(f"Error fetching data for {meeting}: {response.status_code}")
        sleep(1)
    return pd.DataFrame(sessions)

# Pobierz wszystkie sesje 2014–2024
all_sessions_df = get_all_sessions(meetings_key_unique)

Fetched 3 sessions for 1140
Fetched 5 sessions for 1141
Fetched 5 sessions for 1141
Fetched 5 sessions for 1142
Fetched 5 sessions for 1142
Fetched 5 sessions for 1143
Fetched 5 sessions for 1143
Fetched 5 sessions for 1207
Fetched 5 sessions for 1207
Fetched 5 sessions for 1208
Fetched 5 sessions for 1208
Fetched 5 sessions for 1210
Fetched 5 sessions for 1210
Fetched 5 sessions for 1211
Fetched 5 sessions for 1211
Fetched 5 sessions for 1212
Fetched 5 sessions for 1212
Fetched 5 sessions for 1213
Fetched 5 sessions for 1213
Fetched 5 sessions for 1214
Fetched 5 sessions for 1214
Fetched 5 sessions for 1215
Fetched 5 sessions for 1215
Fetched 5 sessions for 1216
Fetched 5 sessions for 1216
Fetched 5 sessions for 1217
Fetched 5 sessions for 1217
Fetched 5 sessions for 1218
Fetched 5 sessions for 1218
Fetched 5 sessions for 1219
Fetched 5 sessions for 1219
Fetched 5 sessions for 1220
Fetched 5 sessions for 1220
Fetched 5 sessions for 1221
Fetched 5 sessions for 1221
Fetched 5 sessions f

In [8]:
all_sessions_df.head()

,meeting_key,year,session_key,session_type,session_name,date_start,time_start,date_end,time_end,gmt_offset,location,country_name,country_code,circuit_short_name
0,1140,2023,9222,Practice,Practice 1,2023-02-23,07:00:00,2023-02-23,16:30:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir
1,1140,2023,7763,Practice,Practice 2,2023-02-24,07:00:00,2023-02-24,16:30:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir
2,1140,2023,7764,Practice,Practice 3,2023-02-25,07:00:00,2023-02-25,16:30:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir
3,1141,2023,7765,Practice,Practice 1,2023-03-03,11:30:00,2023-03-03,12:30:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir
4,1141,2023,7766,Practice,Practice 2,2023-03-03,15:00:00,2023-03-03,16:00:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir


In [36]:
all_sessions_df.tail(100)

,meeting_key,year,session_key,session_type,session_name,date_start,time_start,date_end,time_end,gmt_offset,location,country_name,country_code,circuit_short_name
136,1233,2024,9663,Practice,Practice 1,2024-04-19,03:30:00,2024-04-19,04:30:00,08:00:00,Shanghai,China,CHN,Shanghai
137,1233,2024,9668,Qualifying,Sprint Qualifying,2024-04-19,07:30:00,2024-04-19,08:14:00,08:00:00,Shanghai,China,CHN,Shanghai
138,1233,2024,9672,Race,Sprint,2024-04-20,03:00:00,2024-04-20,03:30:00,08:00:00,Shanghai,China,CHN,Shanghai
139,1233,2024,9664,Qualifying,Qualifying,2024-04-20,07:00:00,2024-04-20,08:00:00,08:00:00,Shanghai,China,CHN,Shanghai
140,1233,2024,9673,Race,Race,2024-04-21,07:00:00,2024-04-21,09:00:00,08:00:00,Shanghai,China,CHN,Shanghai
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,1252,2024,9461,Practice,Practice 1,2024-12-06,09:30:00,2024-12-06,10:30:00,04:00:00,Yas Island,United Arab Emirates,UAE,Yas Marina Circuit
232,1252,2024,9656,Practice,Practice 2,2024-12-06,13:00:00,2024-12-06,14:00:00,04:00:00,Yas Island,United Arab Emirates,UAE,Yas Marina Circuit
233,1252,2024,9657,Practice,Practice 3,2024-12-07,10:30:00,2024-12-07,11:30:00,04:00:00,Yas Island,United Arab Emirates,UAE,Yas Marina Circuit
234,1252,2024,9658,Qualifying,Qualifying,2024-12-07,14:00:00,2024-12-07,15:00:00,04:00:00,Yas Island,United Arab Emirates,UAE,Yas Marina Circuit


In [8]:
sessions_key_unique= all_sessions_df['session_key'].unique()
print(sessions_key_unique)

[9222 7763 7764 7765 7766 7767 7768 7953 7772 7773 7774 7775 7779 7780
 7781 7782 7783 7787 9063 9064 9278 9069 9070 9071 9072 9073 9074 9078
 9087 9088 9089 9090 9094 9095 9096 9097 9098 9102 9103 9104 9105 9106
 9110 9111 9112 9282 9117 9118 9119 9120 9121 9122 9126 9223 9127 9128
 9129 9133 9134 9135 9286 9140 9141 9142 9143 9144 9145 9149 9150 9151
 9152 9153 9157 9158 9159 9160 9161 9165 9166 9167 9168 9169 9173 9214
 9215 9298 9220 9221 9206 9207 9294 9212 9213 9174 9175 9176 9177 9181
 9198 9304 9308 9204 9205 9182 9183 9184 9314 9189 9190 9191 9192 9193
 9197 9462 9463 9464 9465 9466 9467 9468 9472 9473 9474 9475 9476 9480
 9481 9482 9483 9484 9488 9489 9490 9491 9492 9496 9663 9668 9672 9664
 9673 9497 9502 9506 9498 9507 9508 9509 9510 9511 9515 9516 9517 9518
 9519 9523 9524 9525 9526 9527 9531 9532 9533 9534 9535 9539 9540 9545
 9549 9541 9550 9551 9552 9553 9554 9558 9559 9560 9561 9562 9566 9567
 9568 9569 9570 9574 9575 9576 9577 9578 9582 9583 9584 9585 9586 9590
 9591 

In [9]:
# pobranie kierowcow poszczegolnych zespołów dka kazdego z meeting w sessions

def get_all_drivers(unique_meetings_keys):
    drivers = []
    for meeting_key in unique_meetings_keys:
        url = f"https://api.openf1.org/v1/drivers?meeting_key={meeting_key}"
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            
            if isinstance(data, list) and len(data) > 0:
                for driver in data:
                    drivers.append({
                        'driver_number': driver.get('driver_number'),
                        'broadcast_name': driver.get('broadcast_name'),
                        'full_name': driver.get('full_name'),
                        'first_name': driver.get('first_name'),
                        'last_name': driver.get('last_name'),
                        'name_acronym': driver.get('name_acronym'),
                        'country_code': driver.get('country_code'),
                        'team_name': driver.get('team_name'),
                        'meeting_key': meeting_key,
                        'session_key': driver.get('session_key')
                    })
                print(f"Fetched {len(data)} drivers for meeting {meeting_key}")
            else:
                print(f"No drivers found for meeting {meeting_key}")
        else:
            print(f"Error fetching data for meeting {meeting_key}: {response.status_code}")
        sleep(1)
    return pd.DataFrame(drivers)

In [10]:
drivers_df = get_all_drivers(meetings_key_unique)

Fetched 53 drivers for meeting 1140
Fetched 100 drivers for meeting 1141
Fetched 100 drivers for meeting 1141
Fetched 100 drivers for meeting 1142
Fetched 100 drivers for meeting 1142
Fetched 100 drivers for meeting 1143
Fetched 100 drivers for meeting 1143
Fetched 99 drivers for meeting 1207
Fetched 99 drivers for meeting 1207
Fetched 100 drivers for meeting 1208
Fetched 100 drivers for meeting 1208
Fetched 100 drivers for meeting 1210
Fetched 100 drivers for meeting 1210
Fetched 100 drivers for meeting 1211
Fetched 100 drivers for meeting 1211
Fetched 100 drivers for meeting 1212
Fetched 100 drivers for meeting 1212
Fetched 100 drivers for meeting 1213
Fetched 100 drivers for meeting 1213
Fetched 100 drivers for meeting 1214
Fetched 100 drivers for meeting 1214
Fetched 115 drivers for meeting 1215
Fetched 115 drivers for meeting 1215
Fetched 100 drivers for meeting 1216
Fetched 100 drivers for meeting 1216
Fetched 100 drivers for meeting 1217
Fetched 100 drivers for meeting 1217
Fetc

In [12]:
drivers_df.head()

,driver_number,broadcast_name,full_name,first_name,last_name,name_acronym,country_code,team_name,meeting_key,session_key
0,1,M VERSTAPPEN,Max VERSTAPPEN,Max,Verstappen,VER,NED,Red Bull Racing,1140,7763
1,2,L SARGEANT,Logan SARGEANT,Logan,Sargeant,SAR,USA,Williams,1140,7763
2,4,L NORRIS,Lando NORRIS,Lando,Norris,NOR,GBR,McLaren,1140,7763
3,10,P GASLY,Pierre GASLY,Pierre,Gasly,GAS,FRA,Alpine,1140,7763
4,11,S PEREZ,Sergio PEREZ,Sergio,Perez,PER,MEX,Red Bull Racing,1140,7763


In [13]:
unique_drivers = drivers_df['broadcast_name'].unique()

In [14]:
print(unique_drivers)

['M VERSTAPPEN' 'L SARGEANT' 'L NORRIS' 'P GASLY' 'S PEREZ' 'F ALONSO'
 'C LECLERC' 'K MAGNUSSEN' 'N DE VRIES' 'Y TSUNODA' 'G ZHOU'
 'N HULKENBERG' 'E OCON' 'L HAMILTON' 'C SAINZ' 'G RUSSELL' 'O PIASTRI'
 'A ALBON' 'F DRUGOVICH' 'V BOTTAS' 'L STROLL' 'D RICCIARDO' 'P ARON'
 'D BEGANOVIC' "Z O'SULLIVAN" 'L FORNAROLI' 'G BORTOLETO' 'O GOETHE'
 'K FREDERICK' 'G SAUCY' 'N TSOLOV' 'F COLAPINTO' 'M BOYA' 'J EDGAR'
 'S MONTOYA' 'G MINI' 'L BROWNING' 'C COLLET' 'R VILLAGOMEZ' 'T SMITH'
 'O GRAY' 'M ESTERSON' 'I COHEN' 'J MARTI' 'C MANSELL' 'H BARTER'
 'N BEDRIN' 'T BARNARD' 'A GARCIA' 'S FLOERSCH' 'R FARIA' 'W SHIN'
 'R SHWARTZMAN' 'L LAWSON' 'I HADJAR' 'F VESTI' 'O BEARMAN' 'J DOOHAN'
 'T POURCHAIRE' "P O'WARD" 'J DENNIS' 'A IWASA' 'A ANTONELLI' 'R HIRAKAWA'
 'A LECLERC']


In [11]:
# Funkcja do pobierania końcowej tabeli wyników dla sesji
def get_final_session_results(session_keys, include_driver_details=True):
    all_results = []
    
    for session_key in session_keys:
        url = f"https://api.openf1.org/v1/position?session_key={session_key}"
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            
            if isinstance(data, list) and len(data) > 0:
                # Sortuj dane według daty (najnowsze na końcu)
                data_sorted = sorted(data, key=lambda x: x.get('date', ''))
                
                # Znajdź ostatnie pozycje dla każdego kierowcy
                latest_positions = {}
                for record in data_sorted:
                    driver_number = record.get('driver_number')
                    if driver_number:
                        latest_positions[driver_number] = record
                
                date, time = format_datetime_fields(data_sorted[-1]['date'])
                # Dodaj wszystkie końcowe pozycje do wyników
                for driver_number, record in latest_positions.items():
                    result = {
                        'session_key': session_key,
                        'driver_number': driver_number,
                        'position': record.get('position'),
                        'date': date,
                        'time': time,
                        'meeting_key': record.get('meeting_key')
                    }
                    all_results.append(result)
                
                print(f"Session {session_key}: Pobrano {len(latest_positions)} końcowych pozycji")
            else:
                print(f"Session {session_key}: Brak danych pozycji")
        else:
            print(f"Error fetching data for session {session_key}: {response.status_code}")
        
        sleep(0.5)  # Krótka pauza między requestami
    
    results_df = pd.DataFrame(all_results)
    
    if not results_df.empty and include_driver_details:
        # Połącz z danymi o sesjach
        results_df = results_df.merge(
            all_sessions_df[['session_key', 'session_name', 'year', 'location', 'country_name']], 
            on='session_key', 
            how='left'
        )
        
        # Połącz z danymi o kierowcach używając session_key zamiast meeting_key
        results_df = results_df.merge(
            drivers_df[['driver_number', 'session_key', 'broadcast_name', 'full_name', 'team_name']].drop_duplicates(), 
            on=['driver_number', 'session_key'], 
            how='left'
        )
    
    return results_df

In [19]:
results_df = get_final_session_results(sessions_key_unique, include_driver_details=True)
results_df.head()

Session 9222: Pobrano 19 końcowych pozycji
Session 7763: Pobrano 17 końcowych pozycji
Session 7764: Pobrano 17 końcowych pozycji
Session 7765: Pobrano 20 końcowych pozycji
Session 7766: Pobrano 20 końcowych pozycji
Session 7767: Pobrano 20 końcowych pozycji
Session 7768: Pobrano 20 końcowych pozycji
Session 7953: Pobrano 20 końcowych pozycji
Session 7772: Pobrano 20 końcowych pozycji
Session 7773: Pobrano 20 końcowych pozycji
Session 7774: Pobrano 20 końcowych pozycji
Session 7775: Pobrano 20 końcowych pozycji
Session 7779: Pobrano 20 końcowych pozycji
Session 7780: Pobrano 20 końcowych pozycji
Session 7781: Pobrano 20 końcowych pozycji
Session 7782: Pobrano 20 końcowych pozycji
Session 7783: Pobrano 20 końcowych pozycji
Session 7787: Pobrano 20 końcowych pozycji
Session 9063: Pobrano 20 końcowych pozycji
Session 9064: Pobrano 20 końcowych pozycji
Session 9278: Pobrano 20 końcowych pozycji
Session 9069: Pobrano 19 końcowych pozycji
Session 9070: Pobrano 20 końcowych pozycji
Session 907

,session_key,driver_number,position,date,time,meeting_key,session_name,year,location,country_name,broadcast_name,full_name,team_name
0,9222,1,1,2023-02-23,16:13:05.619000,1140,Practice 1,2023,Sakhir,Bahrain,M VERSTAPPEN,Max VERSTAPPEN,Red Bull Racing
1,9222,2,10,2023-02-23,16:13:05.619000,1140,Practice 1,2023,Sakhir,Bahrain,L SARGEANT,Logan SARGEANT,Williams
2,9222,4,5,2023-02-23,16:13:05.619000,1140,Practice 1,2023,Sakhir,Bahrain,L NORRIS,Lando NORRIS,McLaren
3,9222,10,16,2023-02-23,16:13:05.619000,1140,Practice 1,2023,Sakhir,Bahrain,P GASLY,Pierre GASLY,Alpine
4,9222,14,2,2023-02-23,16:13:05.619000,1140,Practice 1,2023,Sakhir,Bahrain,F ALONSO,Fernando ALONSO,Aston Martin


In [20]:
# posortuj wyniki według sesji i pozycji
def sort_results_by_session_and_position(results_df):
    return results_df.sort_values(by=['session_key', 'position'])

sorted_results_df = sort_results_by_session_and_position(results_df)
sorted_results_df.head()

,session_key,driver_number,position,date,time,meeting_key,session_name,year,location,country_name,broadcast_name,full_name,team_name
29,7763,24,1,2023-02-24,16:00:22.519000,1140,Practice 2,2023,Sakhir,Bahrain,G ZHOU,ZHOU Guanyu,Alfa Romeo
19,7763,1,2,2023-02-24,16:00:22.519000,1140,Practice 2,2023,Sakhir,Bahrain,M VERSTAPPEN,Max VERSTAPPEN,Red Bull Racing
24,7763,14,3,2023-02-24,16:00:22.519000,1140,Practice 2,2023,Sakhir,Bahrain,F ALONSO,Fernando ALONSO,Aston Martin
27,7763,21,4,2023-02-24,16:00:22.519000,1140,Practice 2,2023,Sakhir,Bahrain,N DE VRIES,Nyck DE VRIES,AlphaTauri
30,7763,27,5,2023-02-24,16:00:22.519000,1140,Practice 2,2023,Sakhir,Bahrain,N HULKENBERG,Nico HULKENBERG,Haas F1 Team


In [17]:
# Funkcje do analizy końcowych wyników sesji
def get_session_type_results(session_type, max_sessions=None):
    
    sessions = all_sessions_df[all_sessions_df['session_name'] == session_type]
    
    if max_sessions:
        sessions = sessions.head(max_sessions)
    
    session_keys = sessions['session_key'].tolist()
    
    print(f"Pobieranie wyników dla {len(session_keys)} sesji typu '{session_type}'")
    
    return get_final_session_results(session_keys)


In [18]:
# KOMPLETNE POBIERANIE RANKINGU DLA WSZYSTKICH SESJI
# Funkcja do pobierania rankingu dla wszystkich dostępnych sesji każdego typu

def get_all_rankings_by_session_type():

    session_types = all_sessions_df['session_name'].unique()
    all_rankings = {}
    
    
    for session_type in session_types:
        print(f"Przetwarzanie: {session_type}")
        
        # Pobierz wszystkie sesje tego typu
        sessions_of_type = all_sessions_df[all_sessions_df['session_name'] == session_type]
        session_keys = sessions_of_type['session_key'].tolist()
        
        print(f"   Znaleziono {len(session_keys)} sesji typu '{session_type}'")
        
        # Pobierz wyniki dla wszystkich sesji tego typu
        rankings = get_final_session_results(session_keys, include_driver_details=True)
        
        if not rankings.empty:
            # Sortuj według sesji i pozycji
            rankings_sorted = rankings.sort_values(['session_key', 'position'])
            all_rankings[session_type] = rankings_sorted
            
            print(f"Pobrano {len(rankings)} wyników z {rankings['session_key'].nunique()} sesji")
            print(f"Lata: {rankings['year'].min()}-{rankings['year'].max()}")
            print(f"Pozycje: {rankings['position'].min()}-{rankings['position'].max()}")
        else:
            print(f"Brak wyników dla {session_type}")
        
        print()
    
    return all_rankings

In [ ]:
# Zwraca ramkę danych dla każdego meeting ze wszystkimi sesjami i końcowymi pozycjami kierowców

def get_meeting_all_sessions_positions():

    # Pobierz wszystkie session_keys
    all_session_keys = all_sessions_df['session_key'].tolist()
    
    # Pobierz końcowe pozycje ze wszystkich sesji
    all_results = get_final_session_results(all_session_keys, include_driver_details=True)
    
    # Połącz dane meeting z sesjami
    meeting_sessions = all_meetings_df.merge(
        all_sessions_df[['meeting_key', 'session_key', 'session_name', 'session_type']], 
        on='meeting_key', 
        how='inner'
    )
    
    # Połącz z wynikami wszystkich sesji
    final_data = meeting_sessions.merge(
        all_results[['session_key', 'driver_number', 'position', 'broadcast_name', 'full_name', 'team_name']], 
        on='session_key', 
        how='inner'  # inner join żeby mieć tylko te z pozycjami
    )
    
    return final_data

In [ ]:
meeting_all_sessions_positions = get_meeting_all_sessions_positions()
meeting_all_sessions_positions.head()

Session 9222: Pobrano 19 końcowych pozycji
Session 7763: Pobrano 17 końcowych pozycji
Session 7764: Pobrano 17 końcowych pozycji
Session 7765: Pobrano 20 końcowych pozycji
Session 7766: Pobrano 20 końcowych pozycji
Session 7767: Pobrano 20 końcowych pozycji
Session 7768: Pobrano 20 końcowych pozycji
Session 7953: Pobrano 20 końcowych pozycji
Session 7772: Pobrano 20 końcowych pozycji
Session 7773: Pobrano 20 końcowych pozycji
Session 7774: Pobrano 20 końcowych pozycji
Session 7775: Pobrano 20 końcowych pozycji
Session 7779: Pobrano 20 końcowych pozycji
Session 7780: Pobrano 20 końcowych pozycji
Session 7781: Pobrano 20 końcowych pozycji
Session 7782: Pobrano 20 końcowych pozycji
Session 7783: Pobrano 20 końcowych pozycji
Session 7787: Pobrano 20 końcowych pozycji
Session 9063: Pobrano 20 końcowych pozycji
Session 9064: Pobrano 20 końcowych pozycji
Session 9278: Pobrano 20 końcowych pozycji
Session 9069: Pobrano 19 końcowych pozycji
Session 9070: Pobrano 20 końcowych pozycji
Session 907

,circuit_key,circuit_short_name,country_code,country_key,country_name,date_start,time_start,gmt_offset,location,meeting_key,...,meeting_official_name,year,session_key,session_name,session_type,driver_number,position,broadcast_name,full_name,team_name
0,63,Sakhir,BRN,36,Bahrain,2023-02-23,07:00:00,03:00:00,Sakhir,1140,...,FORMULA 1 ARAMCO PRE-SEASON TESTING 2023,2023,9222,Practice 1,Practice,1,1,M VERSTAPPEN,Max VERSTAPPEN,Red Bull Racing
1,63,Sakhir,BRN,36,Bahrain,2023-02-23,07:00:00,03:00:00,Sakhir,1140,...,FORMULA 1 ARAMCO PRE-SEASON TESTING 2023,2023,9222,Practice 1,Practice,2,10,L SARGEANT,Logan SARGEANT,Williams
2,63,Sakhir,BRN,36,Bahrain,2023-02-23,07:00:00,03:00:00,Sakhir,1140,...,FORMULA 1 ARAMCO PRE-SEASON TESTING 2023,2023,9222,Practice 1,Practice,4,5,L NORRIS,Lando NORRIS,McLaren
3,63,Sakhir,BRN,36,Bahrain,2023-02-23,07:00:00,03:00:00,Sakhir,1140,...,FORMULA 1 ARAMCO PRE-SEASON TESTING 2023,2023,9222,Practice 1,Practice,10,16,P GASLY,Pierre GASLY,Alpine
4,63,Sakhir,BRN,36,Bahrain,2023-02-23,07:00:00,03:00:00,Sakhir,1140,...,FORMULA 1 ARAMCO PRE-SEASON TESTING 2023,2023,9222,Practice 1,Practice,14,2,F ALONSO,Fernando ALONSO,Aston Martin


In [43]:
# pobierz początkową pozycję dla każdego z kierowców w meetings dla każdej sesji
def get_starting_session_position(session_keys, include_driver_details=True):
    all_results = []
    
    for session_key in session_keys:
        url = f"https://api.openf1.org/v1/position?session_key={session_key}"
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            
            if isinstance(data, list) and len(data) > 0:
                # Sortuj dane według daty (najstarsze na początku)
                data_sorted = sorted(data, key=lambda x: x.get('date', ''))
                
                # Znajdź pierwsze pozycje dla każdego kierowcy
                first_positions = {}
                for record in data_sorted:
                    driver_number = record.get('driver_number')
                    if driver_number and driver_number not in first_positions:
                        first_positions[driver_number] = record
                
                date, time = format_datetime_fields(data_sorted[0]['date'])
                # Dodaj wszystkie początkowe pozycje do wyników
                for driver_number, record in first_positions.items():
                    result = {
                        'session_key': session_key,
                        'driver_number': driver_number,
                        'position': record.get('position'),
                        'date': date,
                        'time': time,
                        'meeting_key': record.get('meeting_key')
                    }
                    all_results.append(result)
                
                print(f"Session {session_key}: Pobrano {len(first_positions)} początkowych pozycji")
            else:
                print(f"Session {session_key}: Brak danych pozycji")
        else:
            print(f"Error fetching data for session {session_key}: {response.status_code}")
        
        sleep(0.5)  # Krótka pauza między requestami
    
    results_df = pd.DataFrame(all_results)
    
    if not results_df.empty and include_driver_details:
        # Połącz z danymi o sesjach
        results_df = results_df.merge(
            all_sessions_df[['session_key', 'session_name', 'year', 'location', 'country_name']], 
            on='session_key', 
            how='left'
        )
        
        # Połącz z danymi o kierowcach używając session_key zamiast meeting_key
        results_df = results_df.merge(
            drivers_df[['driver_number', 'session_key', 'broadcast_name', 'full_name', 'team_name']].drop_duplicates(), 
            on=['driver_number', 'session_key'], 
            how='left'
        )
    
    return results_df


In [44]:
starting_position_df = get_starting_session_position(sessions_key_unique, include_driver_details=True)
starting_position_df.head()

Session 9222: Pobrano 19 początkowych pozycji
Session 7763: Pobrano 17 początkowych pozycji
Session 7763: Pobrano 17 początkowych pozycji
Session 7764: Pobrano 17 początkowych pozycji
Session 7764: Pobrano 17 początkowych pozycji
Session 7765: Pobrano 20 początkowych pozycji
Session 7765: Pobrano 20 początkowych pozycji
Session 7766: Pobrano 20 początkowych pozycji
Session 7766: Pobrano 20 początkowych pozycji
Session 7767: Pobrano 20 początkowych pozycji
Session 7767: Pobrano 20 początkowych pozycji
Session 7768: Pobrano 20 początkowych pozycji
Session 7768: Pobrano 20 początkowych pozycji
Session 7953: Pobrano 20 początkowych pozycji
Session 7953: Pobrano 20 początkowych pozycji
Session 7772: Pobrano 20 początkowych pozycji
Session 7772: Pobrano 20 początkowych pozycji
Session 7773: Pobrano 20 początkowych pozycji
Session 7773: Pobrano 20 początkowych pozycji
Session 7774: Pobrano 20 początkowych pozycji
Session 7774: Pobrano 20 początkowych pozycji
Session 7775: Pobrano 20 początkow

,session_key,driver_number,position,date,time,meeting_key,session_name,year,location,country_name,broadcast_name,full_name,team_name
0,9222,1,1,2023-02-23,06:54:49.318000,1140,Practice 1,2023,Sakhir,Bahrain,M VERSTAPPEN,Max VERSTAPPEN,Red Bull Racing
1,9222,2,2,2023-02-23,06:54:49.318000,1140,Practice 1,2023,Sakhir,Bahrain,L SARGEANT,Logan SARGEANT,Williams
2,9222,4,3,2023-02-23,06:54:49.318000,1140,Practice 1,2023,Sakhir,Bahrain,L NORRIS,Lando NORRIS,McLaren
3,9222,10,4,2023-02-23,06:54:49.318000,1140,Practice 1,2023,Sakhir,Bahrain,P GASLY,Pierre GASLY,Alpine
4,9222,14,5,2023-02-23,06:54:49.318000,1140,Practice 1,2023,Sakhir,Bahrain,F ALONSO,Fernando ALONSO,Aston Martin


In [49]:
# dla kazdego meeting w session pobierz pit stop dla kazdego kierowcy gdzie pit duration jest rozny od null

def get_pit_stops_for_sessions(session_key):
    pit_stops = []
    
    for session_key in session_key:
        url = f"https://api.openf1.org/v1/pit?session_key={session_key}&pit_duration>0"
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            
            if isinstance(data, list) and len(data) > 0:
                for pit in data:
                    pit_stops.append({
                        'meeting_key': pit.get('meeting_key'),
                        'session_key': session_key,
                        'driver_number': pit.get('driver_number'),
                        'lap': pit.get('lap'),
                        'pit_duration': pit.get('pit_duration'),
                        'date': pit.get('date')
                    })
                print(f"Fetched {len(data)} pit stops for session {session_key}")
            else:
                print(f"No pit stops found for session {session_key}")
        else:
            print(f"Error fetching data for session {session_key}: {response.status_code}")
        
        sleep(0.5)
    
    return pd.DataFrame(pit_stops)


In [50]:
pit_stops_df = get_pit_stops_for_sessions(sessions_key_unique)
pit_stops_df.head()

No pit stops found for session 9222
No pit stops found for session 7763
No pit stops found for session 7763
No pit stops found for session 7764
No pit stops found for session 7764
No pit stops found for session 7765
No pit stops found for session 7765
No pit stops found for session 7766
No pit stops found for session 7766
No pit stops found for session 7767
No pit stops found for session 7767
No pit stops found for session 7768
No pit stops found for session 7768
No pit stops found for session 7953
No pit stops found for session 7953
No pit stops found for session 7772
No pit stops found for session 7772
No pit stops found for session 7773
No pit stops found for session 7773
No pit stops found for session 7774
No pit stops found for session 7774
No pit stops found for session 7775
No pit stops found for session 7775
No pit stops found for session 7779
No pit stops found for session 7779
No pit stops found for session 7780
No pit stops found for session 7780
No pit stops found for sessi

,meeting_key,session_key,driver_number,lap,pit_duration,date
0,1211,9095,14,None,35.3,2023-06-02T11:33:46.930000+00:00
1,1211,9095,18,None,40.8,2023-06-02T11:35:42.475000+00:00
2,1211,9095,21,None,155.2,2023-06-02T11:36:30.849000+00:00
3,1211,9095,2,None,277.0,2023-06-02T11:37:01.583000+00:00
4,1211,9095,16,None,27.6,2023-06-02T11:37:14.911000+00:00


In [58]:
# https://api.openf1.org/v1/race_control
# get all race control for all session in meetings

def get_race_control(session_keys):
    """
    Pobiera wszystkie dane race control dla określonych sesji
    """
    all_race_control = []
    
    for session_key in session_keys:
        url = f"https://api.openf1.org/v1/race_control?session_key={session_key}"
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            
            if isinstance(data, list) and len(data) > 0:
                for control in data:
                    # Formatuj datę i czas
                    date, time = format_datetime_fields(control['date'])
                    
                    race_control_record = {
                        'session_key': session_key,
                        'date': date,
                        'time': time,
                        'category': control.get('category'),
                        'flag': control.get('flag'),
                        'lap_number': control.get('lap_number'),
                        'message': control.get('message')
                    }
                    all_race_control.append(race_control_record)
                
                print(f"Session {session_key}: Pobrano {len(data)} wiadomości race control")
            else:
                print(f"Session {session_key}: Brak danych race control")
        else:
            print(f"Error fetching race control for session {session_key}: {response.status_code}")
        
        sleep(0.5)
    
    return pd.DataFrame(all_race_control)

In [59]:
race_controlls_df = get_race_control(sessions_key_unique[0:5])
race_controlls_df.head()

Session 9222: Pobrano 82 wiadomości race control
Session 7763: Pobrano 36 wiadomości race control
Session 7763: Pobrano 36 wiadomości race control
Session 7764: Pobrano 46 wiadomości race control
Session 7764: Pobrano 46 wiadomości race control
Session 7765: Pobrano 17 wiadomości race control
Session 7765: Pobrano 17 wiadomości race control
Session 7766: Pobrano 15 wiadomości race control
Session 7766: Pobrano 15 wiadomości race control


,session_key,date,time,category,flag,lap_number,message
0,9222,2023-02-23,07:00:00,Flag,GREEN,None,GREEN LIGHT - PIT EXIT OPEN
1,9222,2023-02-23,07:08:43,Flag,RED,None,RED FLAG
2,9222,2023-02-23,07:25:11,Other,None,None,SESSION WILL RESUME AT 10:30
3,9222,2023-02-23,07:25:35,Flag,CLEAR,None,TRACK CLEAR
4,9222,2023-02-23,07:25:39,Drs,None,None,DRS ENABLED


In [ ]:
# pobierz rodzaje opon 
# https://api.openf1.org/v1/stints?session_key={session_key}
#     "compound": "SOFT",
#     "driver_number": 16,
#     "lap_end": 20,
#     "lap_start": 1,
#     "meeting_key": 1219,
#     "session_key": 9165,
#     "stint_number": 1,
#     "tyre_age_at_start": 3

def get_tyre_stints(session_keys):
    all_tyres = []
    
    for session_key in session_keys:
        url = f"https://api.openf1.org/v1/stints?session_key={session_key}"
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            
            if isinstance(data, list) and len(data) > 0:
                for stint in data:
                    tyre_record = {
                        'session_key': session_key,
                        'driver_number': stint.get('driver_number'),
                        'stint_number': stint.get('stint_number'),
                        'compound': stint.get('compound'),
                        'lap_start': stint.get('lap_start'),
                        'lap_end': stint.get('lap_end'),
                        'tyre_age_at_start': stint.get('tyre_age_at_start'),
                        'meeting_key': stint.get('meeting_key')
                    }
                    all_tyres.append(tyre_record)
                
                print(f"Session {session_key}: Pobrano {len(data)} stints")
            else:
                print(f"Session {session_key}: Brak danych o oponach")
        else:
            print(f"Error fetching tyre stints for session {session_key}: {response.status_code}")
        
        sleep(0.5)
    
    return pd.DataFrame(all_tyres)

In [65]:
stints_tyre = get_tyre_stints(sessions_key_unique[0:20])
stints_tyre.head()

Session 9222: Pobrano 267 stints
Session 7763: Pobrano 234 stints
Session 7763: Pobrano 234 stints
Session 7764: Pobrano 271 stints
Session 7764: Pobrano 271 stints
Session 7765: Pobrano 80 stints
Session 7765: Pobrano 80 stints
Session 7766: Pobrano 81 stints
Session 7766: Pobrano 81 stints
Session 7767: Pobrano 81 stints
Session 7767: Pobrano 81 stints
Session 7768: Pobrano 93 stints
Session 7768: Pobrano 93 stints
Session 7953: Pobrano 70 stints
Session 7953: Pobrano 70 stints
Session 7772: Pobrano 79 stints
Session 7772: Pobrano 79 stints
Session 7773: Pobrano 72 stints
Session 7773: Pobrano 72 stints
Session 7774: Pobrano 71 stints
Session 7774: Pobrano 71 stints
Session 7775: Pobrano 80 stints
Session 7775: Pobrano 80 stints
Session 7779: Pobrano 44 stints
Session 7779: Pobrano 44 stints
Session 7780: Pobrano 73 stints
Session 7780: Pobrano 73 stints
Session 7781: Pobrano 68 stints
Session 7781: Pobrano 68 stints
Session 7782: Pobrano 79 stints
Session 7782: Pobrano 79 stints
Ses

,session_key,driver_number,stint_number,compound,lap_start,lap_end,tyre_age_at_start,meeting_key
0,9222,10,1,HARD,1.0,1.0,0,1140
1,9222,27,1,MEDIUM,1.0,1.0,0,1140
2,9222,81,1,HARD,1.0,1.0,0,1140
3,9222,23,1,SOFT,1.0,2.0,0,1140
4,9222,34,1,MEDIUM,1.0,2.0,0,1140


In [67]:
# https://api.openf1.org/v1/weather
    # "air_temperature": 27.8,
    # "date": "2023-05-07T18:42:25.233000+00:00",
    # "humidity": 58,
    # "meeting_key": 1208,
    # "pressure": 1018.7,
    # "rainfall": 0,
    # "session_key": 9078,
    # "track_temperature": 52.5,
    # "wind_direction": 136,
    # "wind_speed": 2.4

def get_weather_data(session_keys):
    all_weather = []
    
    for session_key in session_keys:
        url = f"https://api.openf1.org/v1/weather?session_key={session_key}"
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            
            if isinstance(data, list) and len(data) > 0:
                for weather in data:
                    weather_record = {
                        'session_key': session_key,
                        'meeting_key': weather.get('meeting_key'),
                        'date': format_datetime_fields(weather['date'])[0],
                        'air_temperature': weather.get('air_temperature'),
                        'track_temperature': weather.get('track_temperature'),
                        'humidity': weather.get('humidity'),
                        'pressure': weather.get('pressure'),
                        'rainfall': weather.get('rainfall'),
                        'wind_direction': weather.get('wind_direction'),
                        'wind_speed': weather.get('wind_speed')
                    }
                    all_weather.append(weather_record)
                
                print(f"Session {session_key}: Pobrano {len(data)} rekordów pogody")
            else:
                print(f"Session {session_key}: Brak danych o pogodzie")
        else:
            print(f"Error fetching weather data for session {session_key}: {response.status_code}")
        
        sleep(0.5)
    
    return pd.DataFrame(all_weather)

In [69]:
weather_session_df = get_weather_data(sessions_key_unique[0:20])
weather_session_df.head()

Session 9222: Pobrano 590 rekordów pogody
Session 7763: Pobrano 590 rekordów pogody
Session 7764: Pobrano 581 rekordów pogody
Session 7765: Pobrano 84 rekordów pogody
Session 7766: Pobrano 85 rekordów pogody
Session 7767: Pobrano 86 rekordów pogody
Session 7768: Pobrano 95 rekordów pogody
Session 7953: Pobrano 161 rekordów pogody
Session 7772: Pobrano 82 rekordów pogody
Session 7773: Pobrano 83 rekordów pogody
Session 7774: Pobrano 82 rekordów pogody
Session 7775: Pobrano 80 rekordów pogody
Session 7779: Pobrano 148 rekordów pogody
Session 7780: Pobrano 62 rekordów pogody
Session 7781: Pobrano 82 rekordów pogody
Session 7782: Pobrano 84 rekordów pogody
Session 7783: Pobrano 95 rekordów pogody
Session 7787: Pobrano 222 rekordów pogody
Session 9063: Pobrano 83 rekordów pogody
Session 9064: Pobrano 106 rekordów pogody


,session_key,meeting_key,date,air_temperature,track_temperature,humidity,pressure,rainfall,wind_direction,wind_speed
0,9222,1140,2023-02-23,23.8,29.7,26.0,1012.2,0,193,4.9
1,9222,1140,2023-02-23,23.8,29.7,26.0,1012.2,0,193,4.9
2,9222,1140,2023-02-23,23.8,30.0,27.0,1012.2,0,193,4.3
3,9222,1140,2023-02-23,23.8,30.1,27.0,1012.2,0,186,5.5
4,9222,1140,2023-02-23,23.8,30.1,27.0,1012.2,0,182,4.4
